<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 6.  Vowpal Wabbit

На этой неделе мы познакомимся с популярной библиотекой Vowpal Wabbit и попробуем ее на данных по посещению сайтов.

**План 6 недели:**
- Часть 1. Статья по Vowpal Wabbit
- Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов
 - 2.1. Подготовка данных
 - 2.2. Валидация по отложенной выборке
 - 2.3. Валидация по тестовой выборке (Public Leaderboard)

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. `sklearn.linear_model`. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
Также будет полезна [презентация](https://github.com/esokolov/ml-course-msu/blob/master/ML15/lecture-notes/Sem08_vw.pdf) лектора специализации Евгения Соколова. И, конечно же, [документация](https://github.com/JohnLangford/vowpal_wabbit/wiki) Vowpal Wabbit.

### Задание
1. Заполните код в этой тетрадке 
2. Если вы проходите специализацию Яндеса и МФТИ, пошлите файл с ответами в соответствующем Programming Assignment. <br> Если вы проходите курс ODS, выберите ответы в [веб-форме](https://docs.google.com/forms/d/1wteunpEhAt_9s-WBwxYphB6XpniXsAZiFSNuFNmvOdk).

## Часть 1. Статья про Vowpal Wabbit
Прочитайте [статью](https://habrahabr.ru/company/ods/blog/326418/) про Vowpal Wabbit на Хабре из серии открытого курса OpenDataScience по машинному обучению. Материал для этой статьи зародился из нашей специализации. Скачайте [тетрадку](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_russian/topic08_sgd_hashing_vowpal_wabbit/topic8_sgd_hashing_vowpal_wabbit.ipynb), прилагаемую к статье, посмотрите код, изучите его, поменяйте, только так можно разобраться с Vowpal Wabbit.

## Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов

### 2.1. Подготовка данных

**Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные [отсюда](https://inclass.kaggle.com/c/identify-me-if-you-can4/data) – файлы `train_sessions_400users.csv` и `test_sessions_400users.csv`.**

In [71]:
import os
import pandas as pd
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import numpy as np
import pickle
import re
from vowpalwabbit import pyvw
import csv

In [44]:
!pip install vowpalwabbit

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
!kaggle competitions download -c identify-me-if-you-can4


  0%|          | 0.00/28.9M [00:00<?, ?B/s]
  3%|3         | 1.00M/28.9M [00:00<00:05, 5.37MB/s]
  7%|6         | 2.00M/28.9M [00:00<00:05, 5.31MB/s]
 10%|#         | 3.00M/28.9M [00:00<00:04, 5.65MB/s]
 14%|#3        | 4.00M/28.9M [00:00<00:04, 5.25MB/s]
 17%|#7        | 5.00M/28.9M [00:01<00:05, 4.52MB/s]
 21%|##        | 6.00M/28.9M [00:01<00:05, 4.15MB/s]
 24%|##4       | 7.00M/28.9M [00:01<00:05, 4.08MB/s]
 28%|##7       | 8.00M/28.9M [00:01<00:04, 4.42MB/s]
 31%|###1      | 9.00M/28.9M [00:02<00:04, 4.76MB/s]
 35%|###4      | 10.0M/28.9M [00:02<00:04, 4.78MB/s]
 38%|###8      | 11.0M/28.9M [00:02<00:03, 4.79MB/s]
 41%|####1     | 12.0M/28.9M [00:02<00:03, 4.74MB/s]
 45%|####4     | 13.0M/28.9M [00:02<00:03, 4.82MB/s]
 48%|####8     | 14.0M/28.9M [00:03<00:03, 4.66MB/s]
 52%|#####1    | 15.0M/28.9M [00:03<00:03, 4.27MB/s]
 55%|#####5    | 16.0M/28.9M [00:03<00:03, 4.08MB/s]
 59%|#####8    | 17.0M/28.9M [00:03<00:03, 4.04MB/s]
 62%|######2   | 18.0M/28.9M [00:04<00:02, 4.24MB/s]
 

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'capstone_user_identification'

**Загрузим обучающую и тестовую выборки. Можете заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке. **

In [3]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id')

In [4]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id')

In [5]:
train_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


**Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.**

In [25]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить `LabelEncoder`, да еще и +1 потом добавить (`LabelEncoder` переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.**

In [7]:
y = train_df_400['user_id']
class_encoder = preprocessing.LabelEncoder()
y_for_vw = class_encoder.fit_transform(y)
y_for_vw = y_for_vw + 1

In [8]:
np.min(y_for_vw)

1

**Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:**
- объедините обучающиую и тестовую выборки
- выберите только сайты (признаки от 'site1' до 'site10')
- замените пропуски на нули (сайты у нас нумеровались с 0)
- переведите в разреженный формат `csr_matrix`
- разбейте обратно на обучающую и тестовую части

In [9]:
def make_src(X, vocablen):
    X_sparse = lil_matrix((len(X), vocablen))
    for row, v in enumerate(X):
        for col in v:
            if col != 0:
                X_sparse[row, col-1] += 1

    #X_sparse_toy.todense()
    return X_sparse

In [10]:
sites = ['site' + str(i) for i in range(1, 11)]

In [11]:
with open(os.path.join(PATH_TO_DATA, 'site_dic.pkl'), 'rb') as handler:
    site_vocab = pickle.load(handler)

In [12]:
train_df_400_s = train_df_400[sites].fillna(0).astype('int')
test_df_400_s = test_df_400[sites].fillna(0).astype('int')

In [13]:
train_test_df = pd.concat([train_df_400_s, test_df_400_s])

In [14]:
X_train_test_sparse = make_src(train_test_df.values, len(site_vocab))
X_train_sparse = make_src(train_df_400_s.values, len(site_vocab))
X_test_sparse = make_src(test_df_400_s.values, len(site_vocab))
#y = 

In [26]:
y.head()

session_id
1    653
2    198
3     34
4    601
5    273
Name: user_id, dtype: int64

### 2.2. Валидация по отложенной выборке

**Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.**

In [16]:
# train_share = int(.7 * train_df_400.shape[0])
# train_df_part = train_df_400[sites].iloc[:train_share, :]
# valid_df = train_df_400[sites].iloc[train_share:, :]
# X_train_part_sparse = X_train_sparse[:train_share, :]
# X_valid_sparse = X_train_sparse[train_share:, :]

train_share = int(.7 * train_df_400_s.shape[0])
train_df_part = train_df_400_s[sites].iloc[:train_share, :]
valid_df = train_df_400_s[sites].iloc[train_share:, :]
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [17]:
y_train_part = y[:train_share]
y_valid = y[train_share:]
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

**Реализуйте функцию, `arrays_to_vw`, переводящую обучающую выборку в формат Vowpal Wabbit.**

Вход:
 - X – матрица `NumPy` (обучающая выборка)
 - y (необяз.) - вектор ответов (`NumPy`). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- надо пройтись по всем строкам матрицы `X` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [18]:
train=True
out_file='tmp.vw'

tst = train_df_part.astype(int).to_numpy()[0:2]
tst_y = y_train_part.astype(int).to_numpy()[0:2]
#tst = np.array2string(tst)
output = ''

for i,r in enumerate(tst):
    #i = np.array2string(i)
    if train:
        output += str(tst_y[i]) + ' | '
    else:
        output += str(1) + ' | '
    for c in r:        
        output += str(c) + ' '
    output += '\n'    
print(output)

653 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713 
198 | 8726 8725 665 8727 45 8725 45 5320 5320 5320 



In [35]:
def arrays_to_vw(X, y=None, train=True, out_file='tmp.vw', return_arr = False):
    if type(X) != np.ndarray:
        if type(X) == pd.DataFrame:
            X = X.to_numpy()
    
    if train:
        if type(y) != np.ndarray:
            if type(y) == pd.DataFrame or type(y) == pd.core.series.Series:
                y = y.to_numpy()
                
            
    output = ''
    output_arr = []
    for i,r in enumerate(X):
        #i = np.array2string(i)
        output_str = ''
        if train:
            output_str += str(y[i]) + ' | '
        else:
            output_str += str(1) + ' | '
        for c in r:        
            output_str += str(c) + ' '
        output += output_str + '\n'
        output_arr.append(output_str)

    with open(os.path.join(PATH_TO_DATA, out_file), 'wb') as handler:
        pickle.dump(output, handler)
    
    if return_arr:
        return output_arr

**Примените написанную функцию к части обучащей выборки `(train_df_part, y_train_part_for_vw)`, к отложенной выборке `(valid_df, y_valid_for_vw)`, ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора `NumPy`.**

In [39]:
%%time
# будет 4 вызова
train_part_vw = arrays_to_vw(train_df_part, y_train_part_for_vw, out_file='train_part.vw', return_arr=True)

Wall time: 3.35 s


In [58]:
%%time
valid_vw = arrays_to_vw(valid_df, train=False, out_file='valid.vw', return_arr=True) # y_valid_for_vw

Wall time: 779 ms


In [41]:
%%time
train_full_vw = arrays_to_vw(train_df_400_s, y, out_file='train_full.vw', return_arr=True)

Wall time: 6.31 s


In [42]:
%%time
test_vw = arrays_to_vw(test_df_400_s, train=False, out_file='test.vw', return_arr=True)

Wall time: 660 ms


**Результат должен получиться таким.**

In [7]:
!head -3 $PATH_TO_DATA/train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [8]:
!head -3  $PATH_TO_DATA/valid.vw

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [9]:
!head -3 $PATH_TO_DATA/test.vw

1 | 9 304 308 307 91 308 312 300 305 309
1 | 838 504 68 11 838 11 838 886 27 305
1 | 190 192 8 189 191 189 190 2375 192 8


**Обучите модель Vowpal Wabbitна выборке `train_part.vw`. Укажите, что решается задача классификации с 400 классами (`--oaa`), сделайте 3 прохода по выборке (`--passes`). Задайте некоторый кэш-файл (`--cache_file`, можно просто указать флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента `-k`). Также укажите значение параметра `b`=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажите `random_seed`=17. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.**

In [27]:
# train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
# valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
# train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
# test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
# model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
# pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [ ]:
%%time
!vw -d train_part.vw --oaa 400 --passes 3 --cache_file -b 26 --random_seed 17

In [53]:
model = pyvw.vw(quiet=True, cache_file=True, oaa=400, passes=3, b=26, random_seed=17)

In [54]:
for r in train_part_vw:
    model.learn(r)

**Запишите прогнозы на выборке *valid.vw* в *vw_valid_pred.csv*.**

In [62]:
valid_pred = []
for r in valid_vw:
    valid_pred.append(model.predict(r))

In [70]:
with open ('vw_valid_pred.csv', 'w') as f:
    wr = csv.writer(f)
    wr.writerow(valid_pred)

In [29]:
%%time
!vw ''' ВАШ КОД ЗДЕСЬ '''

Wall time: 53 ms


"vw" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


**Считайте прогнозы *kaggle_data/vw_valid_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [75]:
round(accuracy_score(y_valid_for_vw, valid_pred),3)

0.322

**Теперь обучите `SGDClassifier` (3 прохода по выборке, логистическая функция потерь) и `LogisticRegression` на 70% разреженной обучающей выборки – `(X_train_part_sparse, y_train_part)`, сделайте прогноз для отложенной выборки `(X_valid_sparse, y_valid)` и посчитайте доли верных ответов. Логистическая регрессия будет обучаться не быстро (у меня – 4 минуты) – это нормально. Укажите везде `random_state`=17, `n_jobs`=-1. Для `SGDClassifier` также укажите `max_iter=3`.**

In [76]:
logit = LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit = SGDClassifier(loss='log', max_iter=3, random_state=17, n_jobs=-1)

In [150]:
sgd_logit = SGDClassifier(loss='log', max_iter=3, random_state=17, n_jobs=-1)

In [79]:
%%time
logit.fit(X_train_part_sparse, y_train_part)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Wall time: 19min 32s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2', random_state=17,
                   solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [151]:
%%time
sgd_logit.fit(X_train_part_sparse, y_train_part)

Wall time: 6.21 s


C:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=3,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=17, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

**<font color='red'>Вопрос 1. </font> Посчитайте долю правильных ответов на отложенной выборке для Vowpal Wabbit, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 2. </font> Посчитайте долю правильных ответов на отложенной выборке для SGD, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 3. </font> Посчитайте долю правильных ответов на отложенной выборке для логистической регрессии, округлите до 3 знаков после запятой.**

In [81]:
print('VW: ',round(accuracy_score(y_valid_for_vw, valid_pred),3))

VW:  0.322


In [152]:
print('SGD: ', round(accuracy_score(y_valid, sgd_logit.predict(X_valid_sparse)),3))

SGD:  0.294


In [90]:
print('Logit: ', round(accuracy_score(y_valid, logit.predict(X_valid_sparse)),3))

Logit:  0.363


In [ ]:
vw_valid_acc = ''' ВАШ КОД ЗДЕСЬ '''
sgd_valid_acc = ''' ВАШ КОД ЗДЕСЬ '''
logit_valid_acc = ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [ ]:
write_answer_to_file(round(vw_valid_acc, 3), 'answer6_1.txt')
write_answer_to_file(round(sgd_valid_acc, 3), 'answer6_2.txt')
write_answer_to_file(round(logit_valid_acc, 3), 'answer6_3.txt')

### 2.3. Валидация по тестовой выборке (Public Leaderboard)

**Обучите модель VW с теми же параметрами на всей обучающей выборке – *train.vw*.**

In [92]:
model2 = pyvw.vw(quiet=True, cache_file=True, oaa=400, passes=3, b=26, random_seed=17)

In [93]:
for r in train_full_vw:
    model2.learn(r)

In [94]:
# %%time
# !vw ''' ВАШ КОД ЗДЕСЬ '''

**Сделайте прогноз для тестовой выборки.**

In [137]:
vw_pred = []
for r in test_vw:
    vw_pred.append(model.predict(r))

In [125]:
# %%time
# !vw ''' ВАШ КОД ЗДЕСЬ '''

**Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle.**

In [126]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [138]:
vw_pred = np.array(vw_pred) -1
vw_pred = class_encoder.inverse_transform(vw_pred)

In [139]:
vw_pred = pd.DataFrame({'user_id':vw_pred})

In [140]:
write_to_submission_file(vw_pred, os.path.join(PATH_TO_DATA, 'vw_400_users.csv'))

**Сделайте то же самое для SGD и логистической регрессии. Тут уже ждать обучение логистической регрессии совсем скучно (заново запускать тетрадку вам не захочется), но давайте дождемся.**

In [114]:
logit1 = LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit1 = SGDClassifier(loss='log', max_iter=3, random_state=17, n_jobs=-1)

In [115]:
%%time
logit1.fit(train_df_400_s, y)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Wall time: 2h 44min 45s


C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2', random_state=17,
                   solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [116]:
%%time
sgd_logit1.fit(train_df_400_s, y)

Wall time: 8.4 s


C:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=3,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=17, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [135]:
logit_test_pred = logit.predict(X_test_sparse)
sgd_logit_test_pred = sgd_logit.predict(X_test_sparse)

In [136]:
write_to_submission_file(logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'logit_400_users1.csv'))
write_to_submission_file(sgd_logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'sgd_400_users1.csv'))

Посмотрим на доли правильных ответов на публичной части (public leaderboard) тестовой выборки [этого](https://inclass.kaggle.com/c/identify-me-if-you-can4) соревнования.

**<font color='red'>Вопрос 4. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для Vowpal Wabbit?**

**<font color='red'>Вопрос 5. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для SGD?**

**<font color='red'>Вопрос 6. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для логистической регрессии?**


In [ ]:
vw_lb_score, sgd_lb_score, logit_lb_score = ''' ВАШ КОД ЗДЕСЬ '''

write_answer_to_file(round(vw_lb_score, 3), 'answer6_4.txt')
write_answer_to_file(round(sgd_lb_score, 3), 'answer6_5.txt')
write_answer_to_file(round(logit_lb_score, 3), 'answer6_6.txt')

**В заключение по заданию:**
- Про соотношение качества классификации и скорости обучения VW, SGD и logit выводы предлагается сделать самостоятельно
- Пожалуй, задача классификации на 400 классов (идентификация 400 пользователей) решается недостаточно хорошо при честном отделении по времени тестовой выборки от обучающей. Далее мы будем соревноваться в идентификации одного пользователя (Элис) – [вот](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнование, в котором предлагается поучаствовать. Не перепутайте! 

**Удачи!**